In [12]:
with open('input.txt', 'r', encoding='utf-8')as f :
    text = f.read()

In [13]:
print(f'length of the dataset is {len(text)}')

length of the dataset is 1115394


In [14]:
# All the characters 
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [15]:
# mapping from string to int
stoi = {ch:i for i, ch in enumerate(chars)} # this is the dictionary for string to int 
itos= {i:ch for i , ch in enumerate(chars)} # for i to strings 
encode = lambda s: [stoi[ch] for ch in s ] # encoder gives out the encoded string in form of list 
decode = lambda l: ''.join([itos[i] for i in l]) # decoder gives the decoded sttring

print(encode("Hi Jainit"))
print(decode(encode("hi Jainit")))

[20, 47, 1, 22, 39, 47, 52, 47, 58]
hi Jainit


In [16]:
# now lets bring in the tensors 
import torch 
data = torch.tensor(encode(text), dtype= torch.long)
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [17]:

n = int(0.9*(len(data)))
train_data=data[:n]
val_data = data[n:]

In [18]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [19]:
x = train_data[:block_size]
y= train_data[1:block_size+1]
for t in range (block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when the input is {context}, the target is {target}")

when the input is tensor([18]), the target is 47
when the input is tensor([18, 47]), the target is 56
when the input is tensor([18, 47, 56]), the target is 57
when the input is tensor([18, 47, 56, 57]), the target is 58
when the input is tensor([18, 47, 56, 57, 58]), the target is 1
when the input is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [20]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8 

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split =='train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+block_size+1 ]for i in ix])
    
    return x, y

xb , yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)
print('----')


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


In [29]:
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLM(nn.Module):

    def __init__ (self,vocab_size):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self, idx , targets=None):
        # idx and targets are both (B,T) tensors of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets== None:
            loss =None
        else :
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # print(logits.shape, targets.shape)
            loss = F.cross_entropy(logits , targets )
        return logits, loss 
    
    def generate(self , idx , max_new_tokens):
        # idx is (B,T) array of indices in the current context 
        for _ in range(max_new_tokens):
            # get the preds 
            logits , loss = self(idx)
            logits = logits[:,-1,:]# becomes (B,C)
            probs = F.softmax(logits, dim = 1 ) # get the probabilites 
            idx_next = torch.multinomial(probs, num_samples = 1) # get the top one (B,1)
            idx = torch.cat((idx , idx_next ), dim =1 ) # (B,T+1)
        return idx 
        
m = BigramLM(vocab_size)
logits, loss = m(xb , yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx=torch.zeros((1,1), dtype = torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [30]:
# create a PyTorch optimiser 
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)


In [31]:
batch_size = 32
for steps in range(100):
    xb, yb = get_batch('train')
    logits , loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(loss.item())

4.65630578994751
